In [53]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


In [54]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result


In [55]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

observed_emotions=['calm', 'happy', 'fearful', 'disgust']


In [56]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("E:\Final Project\speech-emotion-recognition-ravdess-data\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)


In [57]:
x_train,x_test,y_train,y_test=load_data(test_size=0.25)


In [58]:
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [59]:
print(f'Features extracted: {x_train.shape[1]}')


Features extracted: 180


In [60]:
model=MLPClassifier(alpha=0.01,batch_size=256,epsilon=1e-08,hidden_layer_sizes=(300,),learning_rate='adaptive',max_iter=800)


In [61]:
model.fit(x_train,y_train)


MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=800)

In [62]:
y_pred=model.predict(x_test)


In [63]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))



Accuracy: 72.92%


In [64]:
print(classification_report(y_test,y_pred)) 
# creating a confusion matrix 
#print(confusion_matrix(y_test, y_pred) )


              precision    recall  f1-score   support

        calm       0.77      0.89      0.83        57
     disgust       0.79      0.65      0.71        48
     fearful       0.63      0.78      0.70        37
       happy       0.71      0.58      0.64        50

    accuracy                           0.73       192
   macro avg       0.73      0.73      0.72       192
weighted avg       0.73      0.73      0.72       192



In [65]:
print("Train data: ", x_train.shape[0], "\nTest data: ", x_test.shape[0])
print(f'Features(Emotions) extracted: {x_train.shape[1]} emotions')


Train data:  576 
Test data:  192
Features(Emotions) extracted: 180 emotions
